In [ ]:
sm = snakemake

In [ ]:
import spherpro.bro as spb
import spherpro.datastore as spd
import spherpro.library as spl
import spherpro.configuration as conf
import spherpro.db as db
import imp
import pycytools as pct
import pycytools.library
import re
import os
import pandas as pd
import numpy as np
import spherpro.library as lib
import matplotlib.pyplot as plt
import matplotlib_scalebar.scalebar as scalebar
import pathlib
import matplotlib.colors as colors
import scipy.ndimage as ndimage

from tqdm import tqdm
%matplotlib inline

# Aim:

Plots for each image number a plot that should allow to easily judge the quality

It should have:

Ir193, Pr141 (tHH3), Pt194, Pt198

Top row: Hm Images

Bottom Row: IMC images

In [ ]:
fn_config = sm.input.fn_config

In [ ]:
fol_plts = pathlib.Path(sm.output[0])
os.makedirs(fol_plts,exist_ok=True)

In [ ]:
bro = spb.get_bro(fn_config)
conf = bro.data.conf

In [ ]:
conf_qd = conf['query_defaults']
conf_cm = conf_qd['channel_measurements']
conf_ob = conf_qd['object_defaults']
conf_dist = conf_qd['distance_measure_raw']

In [ ]:
import spherpro.bromodules.helpers_vz as helpers_vz
V = helpers_vz.VariableBaseHelper


class C:
    PLOT_STACK = conf_cm['stack_name']
    PLOT_MEAS = conf_cm['measurement_name']
    PLOT_OBJECTYPE = conf_qd['object_type']
    PLOT_DISTSTACK = 'DistStack'
    PLOT_DISTMEAS = 'MaxIntensity'
    PLOT_DISTCHAN = 'dist-sphere'
    #PLOT_DISTCHANTYPE = 'Intensity'
    PLOT_DISTTYPE = 'Location'
    


In [ ]:
pannel = bro.doquery(bro.session.query(db.pannel))
pannel

In [ ]:
from spherpro.bromodules.io_stackimage import IoStackImage
iostackimg = IoStackImage(bro)

In [ ]:
def get_validobj_mask(imgid):
    m = bro.io.masks.get_mask(imgid, 'cell')
    obs = get_valid_objs(imgid)
    m_sphere = np.isin(m, obs)
    return m_sphere
    
def get_sphere_outline(mask):
    m = ndimage.filters.maximum_filter(mask, 5)
    m = ndimage.morphology.binary_fill_holes(m)
    return m

def get_valid_objs(imgid):
    return [r[0] for r in bro.session.query(db.objects.object_number)
         .join(db.valid_objects)
         .filter(db.objects.image_id == imgid,
                db.objects.object_type == C.PLOT_OBJECTYPE).all()]

def _plot_imc(channel, imgid, ax=None):
    if ax is None:
        fig = plt.figure(figsize=(20,20))
        ax = plt.gca()
    plane_id = bro.helpers.dbhelp.get_plane_id(C.PLOT_STACK, channel)
    img = iostackimg.get_planeimg(imgid, plane_id)
    ax.imshow(img, cmap='plasma',
                         norm=colors.SymLogNorm(linthresh=1, linscale=0.03, base=10))
    ax.set_title('{} \n {}'.format(bro.helpers.dbhelp.get_target_by_channel(channel), channel))
    ax.axis('off')
    return ax

def _add_mask_contour(mask, ax=None, linewidths=1, linestyles=':', color='white'):
    if ax is None:
        fig = plt.figure(figsize=(20,20))
        ax = plt.gca()
    img = mask.copy()
    ax.contour(img, [0,0.5],linewidths=linewidths, linestyles=linestyles,colors=color)

In [ ]:
imgid= 123

In [ ]:
channels = ['Ir193', 'Pr141', 'Nd144', 'Gd155', 'La139', 'Pt194']

In [ ]:
def check_any_obj(imgid):
    return (bro.session
            .query(db.objects.object_id)
            .filter(db.objects.image_id == imgid,
                   db.objects.object_type == C.PLOT_OBJECTYPE)
            .join(db.valid_objects).first() is not None)

In [ ]:
def plot_qcplot(imgid, channels):
    fig, axs = plt.subplots(2, len(channels)+1)
    m_obj= get_validobj_mask(imgid)
    m_sphere = get_sphere_outline(m_obj)
    for a, c in zip(axs.T, channels):
        bro.plots.heatmask.plt_heatplot([imgid], stat=C.PLOT_MEAS, stack=C.PLOT_STACK, channel=c, ax=a[0], colorbar=False)
        _plot_imc(c, imgid, ax=a[1])
        _add_mask_contour(m_sphere, ax=a[1],linewidths=0.5)
    for ca in a:
        sb = scalebar.ScaleBar(1, units='um', location=4, frameon=False, color='white')
        ca.add_artist(sb)
    a= axs.T[-1]
    bro.plots.heatmask.plt_heatplot([imgid], stat=C.PLOT_DISTMEAS, stack=C.PLOT_DISTSTACK, channel=C.PLOT_DISTCHAN, ax=a[0], colorbar=False)
    _plot_imc('Ir193', imgid, ax=a[1])
    _add_mask_contour(m_obj, ax=a[1], linewidths=0.1)
    plt.suptitle(f'image_id: {imgid}')
    return fig

In [ ]:
bro.plots.heatmask.plt_heatplot([2], stat=C.PLOT_DISTMEAS, stack=C.PLOT_DISTSTACK, channel=C.PLOT_DISTCHAN, colorbar=False)

In [ ]:
def dosave_qcplot(imgid, channels, path):
    fig = plot_qcplot(imgid, channels)
    fn = path / f'qcplot_imgid_{imgid}.png'
    fig.savefig(fn, dpi=300)
    plt.close(fig)

In [ ]:
fig = plot_qcplot(imgid, channels)
fig.set_dpi(300)
#plt.tight_layout()
plt.show()

In [ ]:
valid_imgs = [c[0] for c in bro.session.query(db.valid_images.image_id).all()]

In [ ]:
len(valid_imgs)

In [ ]:
for img in tqdm(sorted(valid_imgs)):
    try:
        dosave_qcplot(int(img), channels, fol_plts)
    except:
        print(f'Error in img {img}')

In [ ]:
for img in tqdm(sorted(valid_imgs)):
    try:
        dosave_qcplot(int(img), channels, fol_plts)
    except:
        print(f'Error in img {img}')

In [ ]:
for img in tqdm(sorted(valid_imgs)):
    try:
        dosave_qcplot(int(img), channels, fol_plts)
    except:
        print(f'Error in img {img}')